# Homework 2.2: The Quest For A Better Network

In this assignment you will build a monster network to solve Tiny ImageNet image classification.

This notebook is intended as a sequel to seminar 3, please give it a try if you haven't done so yet.

(please read it at least diagonally)

* The ultimate quest is to create a network that has as high __accuracy__ as you can push it.
* There is a __mini-report__ at the end that you will have to fill in. We recommend reading it first and filling it while you iterate.
 
## Grading
* starting at zero points
* +20% for describing your iteration path in a report below.
* +20% for building a network that gets above 20% accuracy
* +10% for beating each of these milestones on __TEST__ dataset:
    * 25% (50% points)
    * 30% (60% points)
    * 32.5% (70% points)
    * 35% (80% points)
    * 37.5% (90% points)
    * 40% (full points)
    
## Restrictions
* Please do NOT use pre-trained networks for this assignment until you reach 40%.
 * In other words, base milestones must be beaten without pre-trained nets (and such net must be present in the anytask atttachments). After that, you can use whatever you want.
* you __can't__ do anything with validation data apart from running the evaluation procedure. Please, split train images on train and validation parts

## Tips on what can be done:


 * __Network size__
   * MOAR neurons, 
   * MOAR layers, ([torch.nn docs](http://pytorch.org/docs/master/nn.html))

   * Nonlinearities in the hidden layers
     * tanh, relu, leaky relu, etc
   * Larger networks may take more epochs to train, so don't discard your net just because it could didn't beat the baseline in 5 epochs.

   * Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fhtagn!


### The main rule of prototyping: one change at a time
   * By now you probably have several ideas on what to change. By all means, try them out! But there's a catch: __never test several new things at once__.


### Optimization
   * Training for 100 epochs regardless of anything is probably a bad idea.
   * Some networks converge over 5 epochs, others - over 500.
   * Way to go: stop when validation score is 10 iterations past maximum
   * You should certainly use adaptive optimizers
     * rmsprop, nesterov_momentum, adam, adagrad and so on.
     * Converge faster and sometimes reach better optima
     * It might make sense to tweak learning rate/momentum, other learning parameters, batch size and number of epochs
   * __BatchNormalization__ (nn.BatchNorm2d) for the win!
     * Sometimes more batch normalization is better.
   * __Regularize__ to prevent overfitting
     * Add some L2 weight norm to the loss function, PyTorch will do the rest
       * Can be done manually or like [this](https://discuss.pytorch.org/t/simple-l2-regularization/139/2).
     * Dropout (`nn.Dropout`) - to prevent overfitting
       * Don't overdo it. Check if it actually makes your network better
   
### Convolution architectures
   * This task __can__ be solved by a sequence of convolutions and poolings with batch_norm and ReLU seasoning, but you shouldn't necessarily stop there.
   * [Inception family](https://hacktilldawn.com/2016/09/25/inception-modules-explained-and-implemented/), [ResNet family](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035?gi=9018057983ca), [Densely-connected convolutions (exotic)](https://arxiv.org/abs/1608.06993), [Capsule networks (exotic)](https://arxiv.org/abs/1710.09829)
   * Please do try a few simple architectures before you go for resnet-152.
   * Warning! Training convolutional networks can take long without GPU. That's okay.
     * If you are CPU-only, we still recomment that you try a simple convolutional architecture
     * a perfect option is if you can set it up to run at nighttime and check it up at the morning.
     * Make reasonable layer size estimates. A 128-neuron first convolution is likely an overkill.
     * __To reduce computation__ time by a factor in exchange for some accuracy drop, try using __stride__ parameter. A stride=2 convolution should take roughly 1/4 of the default (stride=1) one.
 
   
### Data augmemntation
   * getting 5x as large dataset for free is a great 
     * Zoom-in+slice = move
     * Rotate+zoom(to remove black stripes)
     * Add Noize (gaussian or bernoulli)
   * Simple way to do that (if you have PIL/Image): 
     * ```from scipy.misc import imrotate,imresize```
     * and a few slicing
     * Other cool libraries: cv2, skimake, PIL/Pillow
   * A more advanced way is to use torchvision transforms:
    ```
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    trainset = torchvision.datasets.ImageFolder(root=path_to_tiny_imagenet, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    ```
   * Or use this tool from Keras (requires theano/tensorflow): [tutorial](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), [docs](https://keras.io/preprocessing/image/)
   * Stay realistic. There's usually no point in flipping dogs upside down as that is not the way you usually see them.
   


In [0]:
import numpy as np
import torch, torchvision
import torch.nn as nn
from torchvision import transforms, datasets
import torch.nn.functional as F
import PIL
import matplotlib.pyplot as plt
import time
import tqdm
import os
%matplotlib inline

In [2]:
from tiny_img import download_tinyImg200
data_path = '.'
download_tinyImg200(data_path)

./tiny-imagenet-200.zip


In [0]:
def create_val_folder(path, filename):
    """
    This method is responsible for separating validation images into separate sub folders
    """
    fp = open(filename, "r")
    data = fp.readlines()

    val_img_dict = {}
    for line in data:
        words = line.split("\t")
        val_img_dict[words[0]] = words[1]
    fp.close()

    for img, folder in val_img_dict.items():
        newpath = (os.path.join(path, folder))
        if not os.path.exists(newpath): 
            os.makedirs(newpath)

        if os.path.exists(os.path.join(path, img)):
            os.rename(os.path.join(path, img), os.path.join(newpath, img))

In [0]:
path = 'tiny-imagenet-200/val/images'
annotations = 'tiny-imagenet-200/val/val_annotations.txt'
create_val_folder(path, annotations)

In [0]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [0]:
transform_test = transforms.Compose([
   torchvision.transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [0]:
dataset = torchvision.datasets.ImageFolder('tiny-imagenet-200/train', transform=transform_train)
test_dataset = torchvision.datasets.ImageFolder('tiny-imagenet-200/val/images', transform=transform_test)

In [0]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [80000, 20000])

In [10]:
print('Train dataset size: {}, Val dataset size: {}'.format(len(train_dataset), len(val_dataset)))

Train dataset size: 80000, Val dataset size: 20000


In [0]:
# Define Hyper parameters
BATCH_SIZE = 128
IMAGE_SIZE = 64
N_CLASSES = 200
N_EPOCHS = 100
LR = 0.001

device = torch.device("cuda:0")

In [0]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [0]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    nn.Conv2d(64, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    nn.MaxPool2d(2),

    nn.Conv2d(64, 128, 3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    nn.Conv2d(128, 128, 3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    nn.MaxPool2d(2),

    nn.Conv2d(128, 256, 3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    nn.Conv2d(256, 256, 3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    nn.Conv2d(256, 256, 3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    nn.MaxPool2d(2),

    nn.Conv2d(256, 512, 3, padding=1),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    nn.Conv2d(512, 512, 3, padding=1),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    nn.Conv2d(512, 512, 3, padding=1),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(512 * 4 * 4, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    # nn.Linear(4096, 4096),
    # nn.ReLU(True),
    # nn.Dropout(),
    nn.Linear(4096, N_CLASSES)
)

In [16]:
model.to(device)

input_tensor = torch.cuda.FloatTensor(1, 3, 64, 64)
output_tensor = model(input_tensor)
print(output_tensor.shape)

torch.Size([1, 200])


In [0]:
opt = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
#  weight_decay=option.l2_reg_weight)
criterion = nn.CrossEntropyLoss()

In [0]:
train_losses, val_losses, val_accs = [], [], []
best_val_acc = 0.0
STATE_DICT_PATH = 'best_weights.pth'

In [19]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Number of trainable parameters: {}'.format(params))

Number of trainable parameters: 42018568


In [0]:
from torch.autograd import Variable
import torch.nn.functional as F


def compute_loss(X_batch, y_batch):
    X_batch = Variable(torch.FloatTensor(X_batch)).cuda()
    y_batch = Variable(torch.LongTensor(y_batch)).cuda()
    logits = model.cuda()(X_batch)
    return F.cross_entropy(logits, y_batch).mean()

In [21]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    print('Epoch {} started... '.format(epoch))
    
    model.train(True)
    epoch_train_losses = []
    for (X_batch, y_batch) in tqdm.tqdm_notebook(train_loader):
        loss = compute_loss(X_batch, y_batch)
        loss.backward()
        
        opt.step()
        opt.zero_grad()
        
        epoch_train_losses.append(loss.cpu().data.numpy())
        
    epoch_train_loss = np.mean(epoch_train_losses)
    print('Train Loss: {}'.format(epoch_train_loss))
    train_losses.append(epoch_train_loss)
    
    model.train(False)
    epoch_val_losses, epoch_val_accs = [], []
    for X_batch, y_batch in tqdm.tqdm_notebook(val_loader):
        
        logits = model(torch.FloatTensor(X_batch).to(device))
        loss = criterion(logits, torch.LongTensor(y_batch).to(device))
        
        y_pred = logits.max(1)[1].data
        
        epoch_val_losses.append(loss.cpu().item())
        epoch_val_accs.append(np.mean( (y_batch.cpu() == y_pred.cpu()).numpy() ))
    
    epoch_val_loss = np.mean(epoch_val_losses)
    epoch_val_acc = np.mean(epoch_val_accs)

    val_losses.append(epoch_val_loss)
    val_accs.append(epoch_val_acc)
    
    print("  Validation loss: \t{:.4f}".format(
        epoch_val_loss))
    print("  Validation accuracy: \t\t\t{:.2f} %".format(
        epoch_val_acc * 100))
    
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        print('Saving model weights....')
        torch.save(model.state_dict(), STATE_DICT_PATH)
    
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, N_EPOCHS, time.time() - start_time))

Epoch 0 started... 



Train Loss: 5.295872211456299



  Validation loss: 	5.0851
  Validation accuracy: 			1.88 %
Saving model weights....
Epoch 1 of 100 took 108.525s
Epoch 1 started... 



Train Loss: 4.947227478027344



  Validation loss: 	4.7743
  Validation accuracy: 			3.93 %
Saving model weights....
Epoch 2 of 100 took 108.502s
Epoch 2 started... 


...


  Validation loss: 	3.7495
  Validation accuracy: 			46.92 %
Epoch 98 of 100 took 108.642s
Epoch 98 started... 



Train Loss: 0.2935165464878082



  Validation loss: 	3.7337
  Validation accuracy: 			47.45 %
Epoch 99 of 100 took 108.799s
Epoch 99 started... 



Train Loss: 0.2880864143371582



  Validation loss: 	3.7143
  Validation accuracy: 			47.45 %
Epoch 100 of 100 took 108.146s


In [22]:
model.load_state_dict(torch.load(STATE_DICT_PATH))

<All keys matched successfully>

In [23]:
print('Best validation accuracy: {}'.format(max(val_accs) * 100))

Best validation accuracy: 48.119028662420384


In [0]:
test_dataset = torchvision.datasets.ImageFolder('tiny-imagenet-200/val/images', transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [25]:
model.train(False)

y_true, y_pred = [], []
for X_batch, y_batch in tqdm.tqdm_notebook(test_loader):

    logits = model(torch.FloatTensor(X_batch).to(device))
    y_predicted = logits.max(1)[1].cpu().numpy()
    
    y_true += list(y_batch.numpy())
    y_pred += list(y_predicted)

When everything is done, please calculate accuracy on `tiny-imagenet-200/val`

In [0]:
from sklearn.metrics import accuracy_score

test_accuracy = accuracy_score(y_true, y_pred)

In [27]:
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_accuracy * 100))

if test_accuracy * 100 > 40:
    print("Achievement unlocked: 110lvl Warlock!")
elif test_accuracy * 100 > 35:
    print("Achievement unlocked: 80lvl Warlock!")
elif test_accuracy * 100 > 30:
    print("Achievement unlocked: 70lvl Warlock!")
elif test_accuracy * 100 > 25:
    print("Achievement unlocked: 60lvl Warlock!")
else:
    print("We need more magic! Follow instructons below")

Final results:
  test accuracy:		47.05 %
Achievement unlocked: 110lvl Warlock!


```

```

```

```

```

```


# Report

All creative approaches are highly welcome, but at the very least it would be great to mention
* the idea;
* brief history of tweaks and improvements;
* what is the final architecture and why?
* what is the training method and, again, why?
* Any regularizations and other techniques applied and their effects;


There is no need to write strict mathematical proofs (unless you want to).
 * "I tried this, this and this, and the second one turned out to be better. And i just didn't like the name of that one" - OK, but can be better
 * "I have analized these and these articles|sources|blog posts, tried that and that to adapt them to my problem and the conclusions are such and such" - the ideal one
 * "I took that code that demo without understanding it, but i'll never confess that and instead i'll make up some pseudoscientific explaination" - __not_ok__

### Hi, my name is `___ ___`, and here's my story

A long time ago in a galaxy far far away, when it was still more than an hour before the deadline, i got an idea:

##### I gonna build a neural network, that
* brief text on what was
* the original idea
* and why it was so

How could i be so naive?!

##### One day, with no signs of warning,
This thing has finally converged and
* Some explaination about what were the results,
* what worked and what didn't
* most importantly - what next steps were taken, if any
* and what were their respective outcomes

##### Finally, after __  iterations, __ mugs of [tea/coffee]
* what was the final architecture
* as well as training method and tricks

That, having wasted ____ [minutes, hours or days] of my life training, got

* accuracy on training: __
* accuracy on validation: __
* accuracy on test: __


[an optional afterword and mortal curses on assignment authors]